# Molecular Optimization in Latent Space with Bayesian Optimization

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sgbaird/honegumi/blob/main/docs/curriculum/tutorials/latent-space/latent-space-optimization.ipynb)

Traditional molecular optimization approaches face significant challenges when dealing with the discrete, combinatorially complex chemical space. Molecules represented as SMILES strings or molecular graphs don't naturally fit into continuous optimization frameworks like Bayesian optimization. However, recent advances in deep generative models, particularly Variational Autoencoders (VAEs), have opened up new possibilities for molecular design.

**Latent space optimization** involves training a generative model (such as a VAE) to encode molecules into a continuous, lower-dimensional latent space where Bayesian optimization can be efficiently performed. The optimized latent representations can then be decoded back into molecular structures.

In this tutorial, we'll demonstrate how to:
1. Encode molecules into a continuous latent space using a simple VAE
2. Perform Bayesian optimization in the latent space
3. Decode optimized latent points back to molecules
4. Optimize molecular properties like drug-likeness (QED score)

---

**Scenario**: You're a computational chemist working on drug discovery, tasked with finding molecules with high drug-likeness while maintaining synthetic accessibility. Traditional approaches would require extensive manual design or brute-force screening of large chemical databases. Instead, you'll use latent space Bayesian optimization to efficiently explore chemical space and identify promising drug candidates.

This approach is particularly valuable for:
- **Drug discovery**: Optimizing ADMET properties, drug-likeness, target binding
- **Materials science**: Designing polymers, catalysts, or electronic materials
- **Sustainable chemistry**: Finding environmentally friendly alternatives

We'll use a simplified molecular VAE and QED (Quantitative Estimate of Drug-likeness) as our objective, but the framework extends to any molecular property prediction model.

## Setup and Dependencies

First, let's install and import the necessary packages. We'll use RDKit for molecular manipulation, scikit-learn for the VAE implementation, and Honegumi/Ax for Bayesian optimization.

In [ ]:
# Install required packages (uncomment if running in Colab)
# !pip install rdkit-pypi scikit-learn matplotlib numpy pandas ax-platform

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')

# Try to import RDKit, use fallback if not available
try:
    from rdkit import Chem
    from rdkit.Chem import Descriptors, QED
    from rdkit.Chem import rdMolDescriptors
    RDKIT_AVAILABLE = True
except ImportError:
    print("RDKit not available, using simplified molecular representations")
    RDKIT_AVAILABLE = False

# Ax platform imports
from ax.service.ax_client import AxClient
from ax.service.utils.instantiation import ObjectiveProperties
from ax.utils.measurement.synthetic_functions import branin
from ax.core.objective import Objective
from ax.core.optimization_config import OptimizationConfig
from ax.core.parameter import RangeParameter, ParameterType
from ax.core.search_space import SearchSpace
from ax.modelbridge.generation_strategy import GenerationStrategy, GenerationStep
from ax.modelbridge.registry import Models

print("Setup complete!")

## Molecular Dataset and Representation

We'll create a simplified molecular dataset. In practice, you might use datasets like ZINC, ChEMBL, or GuacaMol benchmark datasets.

In [ ]:
def generate_molecular_dataset(n_molecules=1000, seed=42):
    """
    Generate a simplified molecular dataset with SMILES and molecular properties.
    If RDKit is available, we use real molecular descriptors. Otherwise, we simulate them.
    """
    np.random.seed(seed)
    
    if RDKIT_AVAILABLE:
        # A small set of drug-like molecules for this example
        base_smiles = [
            "CCO",  # ethanol
            "CC(=O)OC1=CC=CC=C1C(=O)O",  # aspirin
            "CC(C)CC1=CC=C(C=C1)C(C)C(=O)O",  # ibuprofen
            "CN1C=NC2=C1C(=O)N(C(=O)N2C)C",  # caffeine
            "CC(C)(C)C1=CC=C(C=C1)C(C)(C)C",  # simple aromatic
            "CCCCCCCCCCCCCCC(=O)O",  # fatty acid
            "CC1=CC=C(C=C1)N",  # aniline derivative
            "CC(C)C1=CC=C(C=C1)O",  # phenol derivative
        ]
        
        molecules = []
        for i in range(n_molecules):
            # Select a base molecule and add some random variation
            base_smi = np.random.choice(base_smiles)
            mol = Chem.MolFromSmiles(base_smi)
            if mol is not None:
                smiles = Chem.MolToSmiles(mol)
                
                # Calculate molecular descriptors
                mw = Descriptors.MolWt(mol)
                logp = Descriptors.MolLogP(mol)
                tpsa = Descriptors.TPSA(mol)
                qed_score = QED.qed(mol)
                
                molecules.append({
                    'smiles': smiles,
                    'molecular_weight': mw,
                    'logp': logp,
                    'tpsa': tpsa,
                    'qed': qed_score,
                    'mol_id': i
                })
        
        return pd.DataFrame(molecules)
    
    else:
        # Fallback: simulate molecular data
        molecules = []
        for i in range(n_molecules):
            # Simulate molecular descriptors
            mw = np.random.normal(300, 100)  # molecular weight
            logp = np.random.normal(2, 1.5)  # lipophilicity
            tpsa = np.random.exponential(60)  # topological polar surface area
            qed_score = np.random.beta(2, 2)  # QED score between 0 and 1
            
            molecules.append({
                'smiles': f'simulated_mol_{i}',
                'molecular_weight': max(50, mw),
                'logp': logp,
                'tpsa': max(0, tpsa),
                'qed': qed_score,
                'mol_id': i
            })
        
        return pd.DataFrame(molecules)

# Generate the dataset
molecular_data = generate_molecular_dataset(n_molecules=500)
print(f"Generated {len(molecular_data)} molecules")
print("\nDataset preview:")
print(molecular_data.head())
print(f"\nQED score range: {molecular_data['qed'].min():.3f} - {molecular_data['qed'].max():.3f}")

## Molecular VAE: Encoding to Latent Space

Now we'll create a simplified molecular VAE. In practice, you might use more sophisticated models like:
- Junction Tree VAE (JT-VAE)
- Grammar VAE
- Molecular VAE with SMILES
- Graph VAE

For this tutorial, we'll use molecular descriptors as our "encoding" and PCA as a simplified latent space representation.

In [ ]:
class SimplifiedMolecularVAE:
    """
    A simplified molecular VAE using molecular descriptors and PCA.
    In practice, this would be a neural network trained on molecular SMILES or graphs.
    """
    
    def __init__(self, latent_dim=3):
        self.latent_dim = latent_dim
        self.encoder = None  # PCA for dimensionality reduction
        self.decoder = None  # Random Forest for reconstruction
        self.scaler = StandardScaler()
        self.feature_columns = ['molecular_weight', 'logp', 'tpsa']
        
    def fit(self, molecular_data):
        """Train the VAE on molecular data"""
        # Extract molecular features
        features = molecular_data[self.feature_columns].values
        
        # Normalize features
        features_scaled = self.scaler.fit_transform(features)
        
        # Use PCA as the encoder (dimensionality reduction)
        self.encoder = PCA(n_components=self.latent_dim)
        latent_repr = self.encoder.fit_transform(features_scaled)
        
        # Train a decoder (Random Forest) to reconstruct molecular features
        self.decoder = RandomForestRegressor(n_estimators=50, random_state=42)
        self.decoder.fit(latent_repr, features_scaled)
        
        return latent_repr
    
    def encode(self, molecular_data):
        """Encode molecules to latent space"""
        features = molecular_data[self.feature_columns].values
        features_scaled = self.scaler.transform(features)
        return self.encoder.transform(features_scaled)
    
    def decode(self, latent_points):
        """Decode latent points back to molecular features"""
        features_scaled = self.decoder.predict(latent_points)
        features = self.scaler.inverse_transform(features_scaled)
        
        # Convert back to molecular properties
        decoded_molecules = pd.DataFrame(
            features, 
            columns=self.feature_columns
        )
        
        # Ensure realistic ranges
        decoded_molecules['molecular_weight'] = np.clip(decoded_molecules['molecular_weight'], 50, 1000)
        decoded_molecules['tpsa'] = np.clip(decoded_molecules['tpsa'], 0, 200)
        
        return decoded_molecules
    
    def get_latent_bounds(self, molecular_data, percentile=95):
        """Get reasonable bounds for the latent space based on training data"""
        latent_repr = self.encode(molecular_data)
        
        bounds = []
        for i in range(self.latent_dim):
            lower = np.percentile(latent_repr[:, i], 100 - percentile)
            upper = np.percentile(latent_repr[:, i], percentile)
            bounds.append((lower, upper))
        
        return bounds

# Initialize and train the VAE
vae = SimplifiedMolecularVAE(latent_dim=3)
latent_representations = vae.fit(molecular_data)

print(f"Trained VAE with latent dimension: {vae.latent_dim}")
print(f"Latent representations shape: {latent_representations.shape}")
print(f"Explained variance ratio: {vae.encoder.explained_variance_ratio_}")

# Get bounds for latent space optimization
latent_bounds = vae.get_latent_bounds(molecular_data)
print(f"\nLatent space bounds: {latent_bounds}")

## Molecular Property Prediction

We need a function that can predict molecular properties from latent space coordinates. This will serve as our objective function for Bayesian optimization.

In [ ]:
def calculate_qed_from_properties(molecular_props):
    """
    Calculate a simplified QED-like score from molecular properties.
    In practice, you'd use the actual QED calculation or other property predictors.
    """
    if RDKIT_AVAILABLE:
        # If we have RDKit, we'll estimate QED from basic properties
        # This is a simplified approximation
        mw = molecular_props['molecular_weight']
        logp = molecular_props['logp']
        tpsa = molecular_props['tpsa']
        
        # Lipinski's rule of five-inspired scoring
        mw_score = 1.0 if 150 <= mw <= 500 else max(0, 1 - abs(mw - 325) / 325)
        logp_score = 1.0 if -0.4 <= logp <= 5.6 else max(0, 1 - abs(logp - 2.6) / 3.0)
        tpsa_score = 1.0 if tpsa <= 140 else max(0, 1 - (tpsa - 140) / 100)
        
        qed_score = (mw_score * logp_score * tpsa_score) ** (1/3)
        return qed_score
    else:
        # Fallback calculation
        mw = molecular_props['molecular_weight']
        logp = molecular_props['logp']
        tpsa = molecular_props['tpsa']
        
        # Simple scoring function
        score = 0.5 + 0.3 * np.exp(-((mw - 300) / 100) ** 2) + \
                0.2 * np.exp(-((logp - 2) / 1.5) ** 2) + \
                0.1 * np.exp(-(tpsa / 60) ** 2)
        return min(1.0, score)

def molecular_objective_function(latent_point):
    """
    Objective function that takes a latent space point and returns molecular property.
    This function:
    1. Decodes the latent point to molecular features
    2. Calculates molecular properties (QED score)
    3. Returns the property value
    """
    # Reshape if needed
    if len(latent_point.shape) == 1:
        latent_point = latent_point.reshape(1, -1)
    
    # Decode latent point to molecular properties
    decoded_molecules = vae.decode(latent_point)
    
    # Calculate QED score for the first (and only) molecule
    mol_props = decoded_molecules.iloc[0]
    qed_score = calculate_qed_from_properties(mol_props)
    
    return qed_score

# Test the objective function
test_latent = latent_representations[0].reshape(1, -1)
test_score = molecular_objective_function(test_latent)
print(f"Test molecular score: {test_score:.3f}")

## Setting up Bayesian Optimization in Latent Space

Now we'll use Honegumi's Ax integration to perform Bayesian optimization in the latent space. We'll optimize for high QED scores (drug-likeness).

In [ ]:
def ax_objective_function(parameterization):
    """
    Wrapper function for Ax that converts parameterization dict to latent point
    and evaluates the molecular objective.
    """
    # Extract latent coordinates
    latent_point = np.array([
        parameterization['latent_dim_0'],
        parameterization['latent_dim_1'], 
        parameterization['latent_dim_2']
    ])
    
    # Evaluate molecular property
    score = molecular_objective_function(latent_point)
    
    return {"qed_score": (score, 0.0)}  # (value, sem)

# Create search space parameters for latent dimensions
parameters = []
for i, (lower, upper) in enumerate(latent_bounds):
    parameters.append({
        "name": f"latent_dim_{i}",
        "type": "range",
        "bounds": [float(lower), float(upper)],
        "value_type": "float"
    })

# Initialize Ax client
ax_client = AxClient()
ax_client.create_experiment(
    name="molecular_latent_space_optimization",
    parameters=parameters,
    objectives={"qed_score": ObjectiveProperties(minimize=False)}
)

print("Bayesian optimization setup complete!")
print(f"Optimizing over {len(parameters)}D latent space")
print(f"Objective: Maximize QED score (drug-likeness)")

## Initial Training Data from Existing Molecules

Before starting optimization, let's seed the Bayesian optimizer with some known molecules from our dataset.

In [ ]:
# Select a few molecules as initial training data
n_initial = 5
initial_molecules = molecular_data.sample(n=n_initial, random_state=42)
initial_latent = vae.encode(initial_molecules)

print(f"Adding {n_initial} initial molecules to training data:")

for i, (idx, mol) in enumerate(initial_molecules.iterrows()):
    # Create parameter dict for this molecule
    params = {
        'latent_dim_0': initial_latent[i, 0],
        'latent_dim_1': initial_latent[i, 1],
        'latent_dim_2': initial_latent[i, 2]
    }
    
    # Add trial to Ax
    trial_index = ax_client.attach_trial(params)
    ax_client.complete_trial(trial_index=trial_index, raw_data=mol['qed'])
    
    print(f"Molecule {i+1}: QED = {mol['qed']:.3f}, MW = {mol['molecular_weight']:.1f}")

print("\nInitial training data added to Bayesian optimizer.")

## Bayesian Optimization Loop

Now we'll run the main optimization loop, where the Bayesian optimizer suggests new latent space points to evaluate.

In [ ]:
# Run optimization loop
n_iterations = 15
results = []

print(f"Starting Bayesian optimization for {n_iterations} iterations...\n")

for iteration in range(n_iterations):
    # Get next suggested trial
    parameters, trial_index = ax_client.get_next_trial()
    
    # Evaluate the objective function
    result = ax_objective_function(parameters)
    qed_score = result["qed_score"][0]
    
    # Complete the trial
    ax_client.complete_trial(trial_index=trial_index, raw_data=qed_score)
    
    # Decode the latent point to see what molecule we're evaluating
    latent_point = np.array([
        parameters['latent_dim_0'],
        parameters['latent_dim_1'],
        parameters['latent_dim_2']
    ]).reshape(1, -1)
    
    decoded_mol = vae.decode(latent_point).iloc[0]
    
    # Store results
    results.append({
        'iteration': iteration + 1,
        'trial_index': trial_index,
        'qed_score': qed_score,
        'molecular_weight': decoded_mol['molecular_weight'],
        'logp': decoded_mol['logp'],
        'tpsa': decoded_mol['tpsa'],
        'latent_0': parameters['latent_dim_0'],
        'latent_1': parameters['latent_dim_1'],
        'latent_2': parameters['latent_dim_2']
    })
    
    print(f"Iteration {iteration+1:2d}: QED = {qed_score:.3f}, "
          f"MW = {decoded_mol['molecular_weight']:.1f}, "
          f"LogP = {decoded_mol['logp']:.2f}, "
          f"TPSA = {decoded_mol['tpsa']:.1f}")

results_df = pd.DataFrame(results)
print(f"\nOptimization complete! Best QED score: {results_df['qed_score'].max():.3f}")

## Results Analysis

Let's analyze the optimization results and visualize the progress.

In [ ]:
# Plot optimization progress
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 1. QED score over iterations
axes[0, 0].plot(results_df['iteration'], results_df['qed_score'], 'bo-', alpha=0.7)
axes[0, 0].axhline(y=results_df['qed_score'].max(), color='r', linestyle='--', alpha=0.5, label='Best found')
axes[0, 0].set_xlabel('Iteration')
axes[0, 0].set_ylabel('QED Score')
axes[0, 0].set_title('QED Score Optimization Progress')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Molecular weight distribution
axes[0, 1].scatter(results_df['iteration'], results_df['molecular_weight'], 
                   c=results_df['qed_score'], cmap='viridis', alpha=0.7)
axes[0, 1].set_xlabel('Iteration')
axes[0, 1].set_ylabel('Molecular Weight')
axes[0, 1].set_title('Molecular Weight vs QED Score')
cbar1 = plt.colorbar(axes[0, 1].collections[0], ax=axes[0, 1])
cbar1.set_label('QED Score')

# 3. LogP vs TPSA colored by QED
scatter = axes[1, 0].scatter(results_df['logp'], results_df['tpsa'], 
                            c=results_df['qed_score'], cmap='viridis', 
                            s=60, alpha=0.7)
axes[1, 0].set_xlabel('LogP')
axes[1, 0].set_ylabel('TPSA')
axes[1, 0].set_title('Chemical Space Exploration (LogP vs TPSA)')
cbar2 = plt.colorbar(scatter, ax=axes[1, 0])
cbar2.set_label('QED Score')

# 4. Latent space trajectory
axes[1, 1].plot(results_df['latent_0'], results_df['latent_1'], 'bo-', alpha=0.5, markersize=4)
# Color points by QED score
scatter2 = axes[1, 1].scatter(results_df['latent_0'], results_df['latent_1'], 
                             c=results_df['qed_score'], cmap='viridis', s=60, alpha=0.8)
axes[1, 1].set_xlabel('Latent Dimension 0')
axes[1, 1].set_ylabel('Latent Dimension 1')
axes[1, 1].set_title('Optimization Path in Latent Space')
cbar3 = plt.colorbar(scatter2, ax=axes[1, 1])
cbar3.set_label('QED Score')

plt.tight_layout()
plt.show()

# Print best molecules found
best_molecules = results_df.nlargest(3, 'qed_score')
print("\nTop 3 molecules found:")
print("=" * 60)
for i, mol in best_molecules.iterrows():
    print(f"Rank {i+1}: QED = {mol['qed_score']:.3f}")
    print(f"  Molecular Weight: {mol['molecular_weight']:.1f}")
    print(f"  LogP: {mol['logp']:.2f}")
    print(f"  TPSA: {mol['tpsa']:.1f}")
    print(f"  Latent coords: [{mol['latent_0']:.2f}, {mol['latent_1']:.2f}, {mol['latent_2']:.2f}]")
    print()

## Comparison with Random Search

Let's compare our Bayesian optimization results with random search to demonstrate the effectiveness of the approach.

In [ ]:
# Random search comparison
np.random.seed(42)
n_random = len(results_df)
random_results = []

print(f"Running random search with {n_random} evaluations...")

for i in range(n_random):
    # Sample random point in latent space
    random_latent = np.random.uniform(
        low=[bound[0] for bound in latent_bounds],
        high=[bound[1] for bound in latent_bounds],
        size=vae.latent_dim
    )
    
    # Evaluate
    qed_score = molecular_objective_function(random_latent)
    decoded_mol = vae.decode(random_latent.reshape(1, -1)).iloc[0]
    
    random_results.append({
        'iteration': i + 1,
        'qed_score': qed_score,
        'molecular_weight': decoded_mol['molecular_weight'],
        'logp': decoded_mol['logp'],
        'tpsa': decoded_mol['tpsa']
    })

random_df = pd.DataFrame(random_results)

# Plot comparison
plt.figure(figsize=(12, 5))

# Cumulative best
plt.subplot(1, 2, 1)
bo_best = results_df['qed_score'].cummax()
random_best = random_df['qed_score'].cummax()

plt.plot(results_df['iteration'], bo_best, 'b-', label='Bayesian Optimization', linewidth=2)
plt.plot(random_df['iteration'], random_best, 'r-', label='Random Search', linewidth=2)
plt.xlabel('Iteration')
plt.ylabel('Best QED Score Found')
plt.title('Optimization Progress Comparison')
plt.legend()
plt.grid(True, alpha=0.3)

# Distribution comparison
plt.subplot(1, 2, 2)
plt.hist(results_df['qed_score'], bins=10, alpha=0.7, label='Bayesian Optimization', color='blue')
plt.hist(random_df['qed_score'], bins=10, alpha=0.7, label='Random Search', color='red')
plt.xlabel('QED Score')
plt.ylabel('Frequency')
plt.title('Distribution of QED Scores')
plt.legend()

plt.tight_layout()
plt.show()

# Print comparison statistics
print("\nComparison Results:")
print("=" * 40)
print(f"Bayesian Optimization:")
print(f"  Best QED: {results_df['qed_score'].max():.3f}")
print(f"  Mean QED: {results_df['qed_score'].mean():.3f}")
print(f"  Std QED:  {results_df['qed_score'].std():.3f}")

print(f"\nRandom Search:")
print(f"  Best QED: {random_df['qed_score'].max():.3f}")
print(f"  Mean QED: {random_df['qed_score'].mean():.3f}")
print(f"  Std QED:  {random_df['qed_score'].std():.3f}")

improvement = (results_df['qed_score'].max() - random_df['qed_score'].max()) / random_df['qed_score'].max() * 100
print(f"\nImprovement: {improvement:.1f}% better best score with Bayesian optimization")

## Conclusion and Next Steps

In this tutorial, we've demonstrated how to perform Bayesian optimization in molecular latent space. The key advantages of this approach include:

### **What We Accomplished:**
1. **Continuous Representation**: Encoded discrete molecules into a continuous latent space
2. **Efficient Optimization**: Used Bayesian optimization to efficiently explore chemical space
3. **Property Optimization**: Successfully optimized for drug-likeness (QED score)
4. **Meaningful Results**: Found molecules with improved properties compared to random search

### **Real-World Applications:**
- **Drug Discovery**: Optimize ADMET properties, binding affinity, selectivity
- **Materials Science**: Design polymers, catalysts, electronic materials
- **Green Chemistry**: Find environmentally friendly alternatives
- **Lead Optimization**: Improve existing drug candidates

### **Extensions and Improvements:**
1. **Better VAE Models**: Use more sophisticated molecular VAEs (JT-VAE, Grammar VAE, etc.)
2. **Multi-objective Optimization**: Optimize multiple properties simultaneously (e.g., QED + synthetic accessibility)
3. **Constraint Handling**: Add constraints for synthetic feasibility, toxicity, etc.
4. **Active Learning**: Incorporate uncertainty quantification for more efficient exploration
5. **Real Property Predictors**: Use trained models for ADMET, binding affinity, etc.

### **Integration with SDL3 Workflows:**
This approach fits naturally into Self-Driving Labs (SDL3) workflows:
- **Design**: Latent space BO suggests molecular candidates
- **Synthesis**: Automated synthesis of suggested molecules
- **Characterization**: Automated property measurement
- **Learning**: Feedback loop updates the optimization

### **Key Takeaways:**
- Latent space optimization bridges discrete molecular representations with continuous optimization
- Bayesian optimization is significantly more efficient than random search for molecular design
- The framework is flexible and can be adapted to various molecular properties and constraints
- This approach is particularly powerful when combined with experimental automation (SDL3)

The combination of generative models, latent space optimization, and automated experimentation represents a powerful paradigm for accelerating molecular discovery and materials science research.